# Predicting Program Languages

# Goal:

## - Build a classification model that predicts program languages used in each repo on Github.

---

# Imports

In [1]:
import prepare as p




import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import unicodedata
import markdown
import re
import nltk

from bs4 import BeautifulSoup

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

---

# Dictionary

| Column         | Column_type | Data_type| Description              |
|----------------|-------------|----------|--------------------------|
|repo            |Feature      |string    |Name of the repositiory.  |
|language        |Target       |string    |Programming language used.|
|readme_contents |Feature      |string    |Contents for every readme.|

---

# Acquire

In [2]:
df = pd.read_csv('all_repos.csv')
df.head()

,repo,language,readme_contents
0,ScottfreeLLC/AlphaPy,Python,AlphaPy\n=======\n\n|badge_pypi| |badge_downlo...
1,jalapic/engsoccerdata,R,"Latest GitHub version: 11/4/2022, v0.1.7\n\nN..."
2,bttmly/nba,JavaScript,# nba\n*Node.js client for nba.com API endpoin...
3,kyleskom/NBA-Machine-Learning-Sports-Betting,Python,# NBA Sports Betting Using Machine Learning 🏀\...
4,linouk23/NBA-Player-Movements,Python,# NBA Player Movements\n\nThis is a script for...


---

# Wrangle

In [3]:
df = p.actual_languages(df)

/Users/goat/codeup-data-science/NLP-Project/prepare.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['language'].replace(languages_to_rename, 'other')


In [4]:
p.labels_count(df)

,n,percent
other,301,0.386393
Python,241,0.309371
JavaScript,154,0.197689
R,43,0.055199
TypeScript,40,0.051348


---

# Exploration

## What are the most common words in READMEs?

## Does the length of the README vary by programming language?

## Do different programming languages use a different number of unique words?

## Are there any words that uniquely identify a programming language?

## EDA Summary

---

# Pre-Processing

---

# Modeling

## Baseline = 

---

# Conclusion

## Baseline = 

---

# Recomendations

---

# Next Steps

---